In [2]:
import os
import csv
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
def scrape(url):
    content = requests.get(url, allow_redirects=False)
    return BeautifulSoup(content.text, 'html.parser')

In [4]:
year_lst_path = '/Users/Andey/Desktop/Fall2020/CIRP_Lab/jupyter/treaty/treaty_years.csv'
years_df = pd.read_csv(year_lst_path)
all_year_urls = []
for i in range(len(years_df)):
    year = str(years_df.iloc[i]['Years'])
    url = 'https://www.state.gov/'+ year + '-TIAS/?results=100'
    all_year_urls.append(url)

In [5]:
def get_all_treaty_links(url_lst): #gets all treaty links
    all_links = []
    for url in url_lst:
        soup = scrape(url)
        soup.prettify().encode('utf8')
        for a in soup.find_all('a',"collection-result__link", href=True):
            all_links.append(a['href'])
    return all_links

In [6]:
all_treaty_links = get_all_treaty_links(all_year_urls)

In [7]:
all_treaty_links

['https://www.state.gov/antigua_and_barbuda-20-124',
 'https://www.state.gov/chile-20-930',
 'https://www.state.gov/ecuador-20-211',
 'https://www.state.gov/el_salvador-20-302',
 'https://www.state.gov/greece-20-213',
 'https://www.state.gov/guatemala-20-207',
 'https://www.state.gov/guatemala-20-917',
 'https://www.state.gov/honduras-20-325',
 'https://www.state.gov/jordan-20-201',
 'https://www.state.gov/kazakhstan-20-504',
 'https://www.state.gov/netherlands_20-422',
 'https://www.state.gov/rwanda-20-528',
 'https://www.state.gov/saudi_arabia-20-911',
 'https://www.state.gov/shape-20-102',
 'https://www.state.gov/trinidad_and_tobago-20-101',
 'https://www.state.gov/vietnam-20-520',
 'https://www.state.gov/algeria-19-815',
 'https://www.state.gov/argentina-19-314',
 'https://www.state.gov/argentina-19-626',
 'https://www.state.gov/armenia-19-717-agreement-for-cooperation-to-facilitate-the-implementation-of-the-foreign-account-tax-compliance-act/',
 'https://www.state.gov/belgium-19-9

In [15]:
def get_signed_date(text):
    signed_end_idx = text.find(';')
    white_space_count = 0
    signed_start_idx = 0
    for i in range(signed_end_idx, -1, -1): #start date
        if white_space_count == 2 and text[i] == " ":
            signed_start_idx = i
            break
        elif text[i] == " ":
            white_space_count += 1
    signed_date = text[signed_start_idx : signed_end_idx].strip()
    return signed_date

def get_enacted_date(text):
    enacted_end_idx = text.find('.')
    white_space_count = 0
    enacted_start_idx = 0
    for i in range(enacted_end_idx, -1, -1): #start date
        if white_space_count == 2 and text[i] == " ":
            enacted_start_idx = i
            break
        elif text[i] == " ":
            white_space_count += 1
    enacted_date = text[enacted_start_idx : enacted_end_idx].strip()
    return enacted_date

In [16]:
def get_all_prez_links():
    all_presidents_url = 'https://www.presidency.ucsb.edu/presidents'
    soup = scrape(all_presidents_url)
    soup.prettify().encode('utf8')
    content = soup.find_all("div", {"class": "views-field views-field-title"})
    all_links = []
    for i in content:
        link = "https://www.presidency.ucsb.edu" + str(i.find("a").get("href"))
        all_links.append(link)
    return all_links

In [17]:
all_prez_url = get_all_prez_links()

In [18]:
# from datetime import datetime

# now = datetime.now()
# now

In [19]:
import datetime
# from datetime import datetime
def normalize_prez_dates(s):
    idx = s.find(' to ')
    start_date = s[:idx]
#     time_object = start_date.format(datetime.datetime.now())
#     time_object = datetime.datetime.strptime(start_date, '%B %d %Y')
#     start_date = start_date.date
    end_date = s[idx+4:]
    return (start_date, end_date)

In [20]:
def dict_of_all_presidents(all_prez_url):
    for url in all_prez_url:
        soup = scrape(url)
        soup.prettify().encode('utf8')
        name = soup.find("div", {"class": "field-title"}).get_text().strip()
        dates = soup.find("span", {"class": "date-display-range"}).get_text().strip()
        start_date, end_date = normalize_prez_dates(dates)
        party = soup.find("div", {"class": "f-item"}).get_text().strip()
        print(party)
        break
dict_of_all_presidents(all_prez_url)

January 20, 2017 to January 20, 2021


In [22]:
def dict_of_all_presidents():#maps dict of list of presidents to id_number
    all_presidents_url = 'https://www.presidency.ucsb.edu/presidents'
    soup = scrape(all_presidents_url)
    soup.prettify().encode('utf8')
    texts = soup.findAll(text=True)
    start_index = texts.index('Donald J. Trump')
    end_index = texts.index('George Washington')
    texts = texts[start_index: end_index+1]
    print(texts)
    dict_all_prezs = dict()
    count = 45
    for elem in texts:
#         years = 
        if not elem.isdigit() and elem != ' to ' and elem != ' ' and elem != '\n':
            dict_all_prezs[elem] = count
            count -= 1
    return dict_all_prezs

In [27]:
#create empty df
cols = ['name', 'signed_date', 'enacted_date', 'website_url', 'treaty_url']
index = []
for i in range(len(all_treaty_links)):
    index.append(i)
treaty_df = pd.DataFrame(index=index, columns=cols)
treaty_df = treaty_df.fillna(0)

In [51]:
def populate_treaty_df():
    index = 0
    for treaty_link in all_treaty_links:
        soup = scrape(treaty_link)
        soup.prettify().encode('utf8')
        if soup.find("h1", {"class": "featured-content__headline stars-above"}) == None:
            name = "None"
        else:
            name = soup.find("h1", {"class": "featured-content__headline stars-above"}).get_text().strip()
        if soup.find_all('a',"button button--outline button--outline-red button--corners button--download", href=True) == None:
            link = "None"
        else:
            for a in soup.find_all('a',"button button--outline button--outline-red button--corners button--download", href=True):
                link = a['href']
        if soup.find("div", {"class": "entry-content"}) == None:
            enacted_date, signed_date = "None", "None"
        else:
            content = soup.find("div", {"class": "entry-content"}).get_text().strip()
            signed_date = get_signed_date(content)
            enacted_date = get_enacted_date(content)
        treaty_df.loc[index, 'name'] = name
        treaty_df.loc[index, 'signed_date'] = signed_date
        treaty_df.loc[index, 'enacted_date'] = enacted_date
        treaty_df.loc[index, 'website_url'] = treaty_link
        treaty_df.loc[index, 'treaty_url'] = link
        index += 1
        print(index, name)

1327 Paraguay (06-801) – Air Transport Agreement
1328 Peru (06-1025.1) – Agreement on Beef SPS Issues for the United States-Peru Trade Promotion
1329 Peru (06-105.1) – Agreement Regarding Sanitary and Phytosanitary Measures and Technical Barriers to Trade
1330 Peru (06-301.1) – Agreement Regarding Mutual Assistance Between Their Customs Administrations
1331 Peru (06-410.1) – Agreement Relating to the United States-Peru Trade Promotion Agreement Regarding Food Safety
1332 Philippines (06-412.1) – Defense Agreement Regarding the United States Forces Visiting the Philippines Establishing a Security Engagement Board
1333 Philippines (06-502.1) – Defense Cooperation Regarding Defense Articles
1334 Poland (06-1122) – Agreement for Scientific and Technical Cooperation
1335 Romania (06-721) – Agreement Regarding the Activities of United States Forces Located on the Territory of Romania
1336 Russian Federation (06-1119.1) – Agreement on Agricultural Biotechnology
1337 Russian Federation (06-111

1397 Mexico (05-727) – Protocol Concerning the Allotment and Use of the 406.1 – 420 Band for Fixed and Mobile Services Along the Common Border
1398 Mexico (05-727.2) – Protocol Concerning the Allotment and Use of the 380-399.9 MHz Band for Fixed and Mobile Terrestrial Non-Broadcasting Services Along the Common Border
1399 Mongolia (05-1119) – Agreement Concerning the International School of Ulaanbaatar
1400 Mozambique (05-1007) – Agreement Concerning Reciprocal and Property Ownership Rights for the Diplomatic and Consular Missions of the United States and Mozambique
1401 Mozambique (13006) – Treaty Concerning the Encouragement and Reciprocal Protection of Investment
1402 Multilateral (05-1002) – Revised International Plant Protection Convention
1403 Multilateral (05-110) – Framework Memorandum of Understanding Concerning Cooperation in Post Production Support of Harrier Aircraft
1404 Multilateral (05-1202) – Memorandum of Understanding Concerning Projects for the Airborne Early Warning

1470 Dominica (04-0510) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1471 Dominican Republic (04-0812) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1472 El Salvador (04-0621) – Agreement Regarding the Transfer of Persons to the International Criminal Court
1473 El Salvador (04-1217) – Agreement Regarding the Cooperating Nation Information Exchange System
1474 Equatorial Guinea (04-0506) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1475 Eritrea (04-0708) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1476 Estonia (04-0810) – Protocol to the Treaty of April 19, 1994 for the Encouragement and Reciprocal Protection of Investment
1477 European Community (04-0422) – Agreement on Intensifying and Broadening the Agreement on Customs Cooperation and Mutual Assistance in Customs Matters to Include Cooperation in Container Security and Related Matte

1539 Albania (03-1009) – Agreement Concerning the Program of the Peace Corps in Albania
1540 Albania (03-707.2) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1541 Antigua and Barbuda (03-929) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1542 Antigua and Barbuda (03-930) – Protocol to the Agreement of April 19, 1995, as Amended, Concerning Maritime Counter-Drug Operations
1543 Antigua and Barbuda (13178) – Agreement for the Exchange of Information with Respect to Taxes
1544 Australia (03-423) – Memorandum of Understanding for Test and Evaluation Program (TEP) Cooperation
1545 Australia (13164) – Protocol Amending the Convention of August 6, 1982 for the Avoidance of Double Taxation and the Prevention of Fiscal Evasion with Respect to Taxes on Income
1546 Azerbaijan (03-507) – Agreement Concerning Cooperation in the Area of Counterproliferation of Weapons of Mass Destruction and Defense Activities
1547 Azerbaijan 

1613 Malawi (03-923) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1614 Maldives (03-708) – Agreement Regarding the Surrender of Persons to International Tribunals
1615 Marshall Islands (03-626.1) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1616 Mauritania (03-706) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1617 Mauritius (03-630.2) – Agreement Regarding the Surrender of Persons to the International Criminal Court
1618 Mexico (03-1001.1) – Minute 310 of the International Boundary Water Commission: Emergency Delivery of Colorado River Water for Use in Tijuana, Baja California
1619 Mexico (03-1112.1) – Agreement to Establish a Program of the Peace Corps in Mexico
1620 Mexico (03-703) – Second Protocol to Convention Regarding Taxation
1621 Mexico (03-707) – Minute 309 of the International Boundary Water Commission: Volumes of Water Saved with the Modernization and Improved T

...

1977 New Zealand (13034) – Agreement Relating to the Employment of Dependents of Official Government Employees
1978 Nicaragua (99-310) – Memorandum of Understanding Concerning Scientific Cooperation in the Earth and Mapping Sciences
1979 Organization for the Prohibition of Chemical Weapons (13022) – Tax Reimbursement Agreement
1980 Pakistan (12851) – Air Transport Agreement
1981 Paraguay (99-913) – Memorandum of Understanding Concerning the Operation of a Seismic Monitoring Station in Paraguay
1982 Philippines (12931) – Agreement Regarding the Treatment of US Armed Forces Visiting the Philippines; Agreement Regarding the Treatment of Philippines Personnel Visiting the US
1983 Philippines (13018) – Agreement for Cooperation in the Global Learning and Observations to Benefit the Environment (GLOBE) Program
1984 Poland (99-917) – Treaty on Extradition
1985 Poland (99-917.1) – Treaty on Mutual Legal Assistance
1986 Romania (12974) – Air Transport Agreement
1987 Romania (12976) – Agreement 

2055 Japan (13004) – Agreement Concerning Cooperation on the Advanced Microwave Scanning Radiometer-E (AMSR-E) Program
2056 Japan (98-620) – Protocol Extending the Agreement of June 20, 1988, as Extended, on Cooperation in Research and Development in Science and Technology
2057 Kenya (13008) – Investment Incentive Agreement
2058 Korea (12961) – Air Transport Agreement
2059 Korea (12980) – Investment Incentive Agreement
2060 Latvia (12922) – Agreement Concerning Security Measures for the Protection of Classified Military Information
2061 Mauritius (12912) – Investment Incentive Agreement
2062 Mexico (12907) – Minute 298 of the International Boundary and Water Commission: Recommendations for Construction of Works Parallel to the City of Tijuana, B.C.
2063 Mexico (12908) – Agreement Amending the Air Transport Agreement of August 15, 1960, as Amended and Extended
2064 Mexico (13009) – Memorandum of Understanding Concerning the Use of Radio Frequencies, Coordination and Cooperation for Emer

2129 Guatemala (12909) – Agreement for Cooperation in the Global Learning and Observations to Benefit the Environment (GLOBE) Program
2130 Guatemala (97-929) – Memorandum of Understanding Concerning the Imposition of Import Restrictions on Archaeological Objects and Material from the Pre-Columbian Cultures of Guatemala
2131 Haiti (12880) – Memorandum of Agreement Concerning the Provision of Civil Aviation Assistance
2132 Honduras (12896) – Agreement for Cooperation in the Global Learning and Observations to Benefit the Environment (GLOBE) Program
2133 Hong Kong (12849) – Agreement Concerning Air Services
2134 Hungary (97-318) – Treaty on Extradition
2135 Hungary (97-318.1) – Treaty on Mutual Legal Assistance in Criminal Matters
2136 International Organization for Migration (12883) – Tax Reimbursement Agreement
2137 Ireland (12831) – Agreement for Promotion of Aviation Safety
2138 Ireland (12877) – Agreement Relating to the Employment of Dependents of Official Government Employees
2139 

2206 Ghana (12817) – Agreement Regarding the Provision of Commodities, Services and Associated Military Education and Training to Ghanaian Forces Participating in ECOMOG Peacekeeping Operations
2207 Greece (12848) – Agreement Amending the Defense Industrial Cooperation Agreement of November 10, 1986
2208 Hungary (12822) – Mutual Logistic Support Agreement
2209 Indonesia (96-1223) – Protocol Amending the Convention of July 11, 1988 for the Avoidance of Double Taxation and the Prevention of Fiscal Evasion with Respect to Taxes on Income
2210 Japan (12793) – Agreement Relating to and Amending the Civil Air Transport Agreement of August 11, 1952, as Amended
2211 Japan (12802) – Agreement Concerning a Program for the Cooperative Research of Eyesafe Laser Radar
2212 Japan (12809) – Agreement Concerning Cooperation on the Advanced Spaceborne Thermal Emission and Reflection Radiometer (ASTER) Program
2213 Jordan (12810) – Agreement Concerning the Program of the Peace Corps in Jordan
2214 Jorda

2291 Chad (92-623.1) – Employment Agreement
2292 Egypt (92-627) – Treaty Concerning the Reciprocal Encouragement and Protection of Investments with Supplementary Protocol
2293 Greece (12878) – Air Transport Agreement
2294 Ireland (92-1118) – Protocol to Trade and Investment Treaty of January 21, 1950
2295 Kyrgyzstan (92-821) – Agreement on Trade Relations with Exchanges of Letters
2296 Lithuania (92-1112) – Agreement Concerning the Development of Trade and Investment Relations
2297 Moldova (92-702) – Agreement on Trade Relations
2298 Mongolia (92-327) – Memorandum of Understanding Concerning Property of Diplomatic Missions
2299 Multilateral (92-228) – Convention for the Prohibition of Fishing with Long Driftnets in the South Pacific with Protocol
2300 Multilateral (92-908) – International Covenant on Civil and Political Rights
2301 New Zealand (92-1002) – Agreement Concerning a Framework of Principles and Procedures for Consultations Regarding Trade and Investment Relations
2302 Ukrain

In [56]:
treaty_df

,name,signed_date,enacted_date,website_url,treaty_url
0,Antigua and Barbuda (20-124) – Status of Force...,"March 12, 2014","January 24, 2020",https://www.state.gov/antigua_and_barbuda-20-124,https://www.state.gov/wp-content/uploads/2020/...
1,Chile (20-930) – Memorandum of Understanding C...,"May 7, 2020","September 30, 2020",https://www.state.gov/chile-20-930,https://www.state.gov/wp-content/uploads/2020/...
2,Ecuador (20-211) – Memorandum of Understanding...,"May 22, 2019","February 11, 2020",https://www.state.gov/ecuador-20-211,https://www.state.gov/wp-content/uploads/2020/...
3,El Salvador (20-302) – Memorandum of Understan...,"March 2, 2020","March 2, 2020",https://www.state.gov/el_salvador-20-302,https://www.state.gov/wp-content/uploads/2020/...
4,Greece (20-213) – Amendment of Agreement of Ju...,"October 5, 2019","February 13, 2020",https://www.state.gov/greece-20-213,https://www.state.gov/wp-content/uploads/2020/...
...,...,...,...,...,...
2350,Italy (85-1113) – Treaty on Mutual Legal Assis...,"November 9, 1982",1984 (Treaty Doc,https://www.state.gov/85-1113/,https://www.state.gov/wp-content/uploads/2019/...
2351,Liberia (84-1016) – Employment Agreement,"and 16, 1984","October 16, 1984",https://www.state.gov/84-1016,https://www.state.gov/wp-content/uploads/2019/...
2352,Multilateral (84-229) – Revised Agreement Conc...,"May 13, 1977",1979 (Treaty Doc,https://www.state.gov/84-229/,https://www.state.gov/wp-content/uploads/2019/...
2353,Multilateral (83-1202) – Convention on the Own...,"November 14, 1970",1972 (Treaty Doc,https://www.state.gov/83-1202/,https://www.state.gov/wp-content/uploads/2019/...


In [54]:
# treaty_df.to_csv('/Users/Andey/Desktop/Fall2020/CIRP_Lab/all_csv_files/treaties.csv')